# 동영상 오디오 데이터를 통해 키워드 추출

In [22]:
import yt_dlp

def download_audio(youtube_url, output_file="audio_new.mp3"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_file
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

# 사용 예시
download_audio("https://www.youtube.com/watch?v=szmtkszxHBg&t=2039s")


[youtube] Extracting URL: https://www.youtube.com/watch?v=szmtkszxHBg&t=2039s
[youtube] szmtkszxHBg: Downloading webpage
[youtube] szmtkszxHBg: Downloading tv client config
[youtube] szmtkszxHBg: Downloading player 91201489
[youtube] szmtkszxHBg: Downloading tv player API JSON
[youtube] szmtkszxHBg: Downloading ios player API JSON
[youtube] szmtkszxHBg: Downloading m3u8 information
[info] szmtkszxHBg: Downloading 1 format(s): 251
[download] Destination: audio_new.mp3
[download] 100% of   30.05MiB in 00:00:10 at 2.78MiB/s     
[ExtractAudio] Destination: audio_new.mp3.mp3
Deleting original file audio_new.mp3 (pass -k to keep)


In [35]:
import whisper

def transcribe_audio(audio_path):
    model = whisper.load_model("small")  # 모델 크기: "tiny", "base", "small", "medium", "large"
    result = model.transcribe(audio_path)
    return result["text"]

# 변환 실행
transcript = transcribe_audio("audio_new.mp3")
# transcript를 txt 파일로 저장
with open("transcript.txt", "w", encoding="utf-8") as file:
    file.write(transcript)

print(transcript)


100%|███████████████████████████████████████| 461M/461M [01:12<00:00, 6.68MiB/s]
c:\Users\jhkan\workspace\howabout\youtube\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 안녕하세요 동인간 뉴스터지 동정민 앤컴 입니다. 박사님들 어서 오십시오. 윤석열 대통령이 겸을 왜 했을까 상당히 처음부터 많은 궁금증이 있었는데 헌법 재판소 변론 때 2차 변론이었는데 사회상 이게 이제 본격적인 첫 변론이었어요. 이때 대통령치게 데리는다는 배진한 변호사가 도대체 왜 국가 비상 사태로 그때 판단할 수밖에 없었는지 이유를 설명하겠다며 첫 번째로 이걸 들었습니다. 바로 최대 국정물난 사태인 부정선거 의혹. 우리나라에서 판돌아예 상자 같은 걸까요? 한쪽에서는 뭔가 이 안에 뭐가 있다. 확실한 사실로 믿고 있는 세력이 있고 한쪽은 완전히 이 부정선거 의혹만 얘기하면 이거는 음모로네 빠져있는 거다라고 주장하는 완전히 갈리는 이슈죠. 윤석열 대통령은 부정선거에 확실하게 의심을 갖고 있습니다. 대통령이 최포되던 날 공개한 자기가 자필로 쓴 내용을 보면 여기 분명히 적혀 있죠. 우리나라 선거에서 부정선거의 증거는 너무나 많습니다. 이를 가능하게 하는 선거의 엉터리 시스템도 다 드러났습니다. 부정선거를 음모론으로 일축할 수 없습니다. 이번에 헌재에 나와서도 이렇게 얘기했습니다. 저는 선거가 부정이어서 믿을 수 없다 이 음모론이 아니라 뭔가 팩트를 확인하는 게 필요했습니다. 그러면서요. 제가 이걸 살펴볼 겁니다. 박 쌤들. 아, 대체 부정선거 의혹. 뭐 의혹은 있는데 이게 대체 무슨 의혹인 건지. 그럼 이게 정말 믿을만 한 건지. 그럼 선거는 뭐라고 해명을 하고 있는 건지. 제대로 해명을 하고 있는 건지. 너무 궁금하잖아요. 과연 대통령은 부정선거와 관련해서 무엇을 의심하고 있는가가 너무 궁금했어요. 무엇. 대체 무엇을 의심하고 있는가 처음으로 밝혔습니다. 대통령 측에서요. 부정선거 우리는 이걸 의심합니다라고 10가지의 의심하는 사유를 밝혔습니다. 헌법 재판소에서. 그래서 제가 이 10가지를 선거님께 제가 물어봤습니다. 답을 해보십시오. 대통령 측에서는 이렇게 10가지를 의심한다는 데 선거님 여러분은 어떻게 해명을 하시겠습니까. 그 답을 들어왔습니다. 자, 박사

In [28]:
with open("transcript.txt", "r", encoding="utf-8") as file:
    transcript = file.read()


In [ ]:
import openai

# OpenAI API 키 설정
openai.api_key = "MY_OPENAPI_KEY"

def extract_keywords_gpt(text):
    """
    GPT-4를 이용하여 텍스트에서 핵심 키워드를 추출하는 함수
    """
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # 최신 모델 사용
        messages=[
            {"role": "system", "content": "너는 유튜브 동영상 내용을 분석하여 키워드를 추출하는 AI야."},
            {"role": "user", "content": f"이 텍스트에서 핵심 키워드를 뽑아줘: {text}"}
        ],
        temperature=0.5,  # 창의성 조절 (0에 가까울수록 일관된 답변)
        max_tokens=100  # 응답 길이 제한
    )

    # 응답에서 키워드 리스트 추출
    return response.choices[0].message.content

# 테스트 실행
keywords = extract_keywords_gpt(transcript)
print("GPT 키워드:", keywords)


GPT 키워드: 주요 키워드: 윤석열 대통령, 부정선거 의혹, 국정원, 선관위, 사전투표, 투표 용지, 보안검사, 헤킹, 개표 시스템, 보완관리체계


In [25]:
import tiktoken

# OpenAI의 토큰 계산을 위해 'cl100k_base' 인코더 사용 (GPT-3.5 및 GPT-4에서 사용됨)
encoding = tiktoken.get_encoding("cl100k_base")

# 파일 읽기
file_path = "transcript.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_content = file.read()

# 토큰 개수 계산
token_count = len(encoding.encode(text_content))
token_count

15556

In [31]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer


def extract_keywords_bert(text, top_n=10):
    kw_model = KeyBERT(SentenceTransformer("paraphrase-mpnet-base-v2"))
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)
    return [word for word, score in keywords]

# 파일 읽기
file_path = "transcript.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

# 키워드 추출 실행
keywords = extract_keywords_bert(text)
print("BERT 기반 키워드:", keywords)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BERT 기반 키워드: ['안녕하세요 동인간유스터지', '중앙선거관리원에 함께', '인세의 무방비더라라고', '성관이의 설명이에요', '국정원의 보안정검이', '알려주셔서 청찰려들이', '당첨자님은 안경님이이신데요', '마찬가지죠 성관이에서는', '국정원의 보안정검에', '선관이의 설명입니다']


In [33]:
from transformers import pipeline

# 무료 GPT 대체 모델 (Mistral 사용)
model_name = "mistralai/Mistral-7B-v0.1"
llm = pipeline("text-generation", model=model_name, device="cpu")

def extract_keywords_mistral(text):
    prompt = f"Extract the 10 most important keywords from the following text:\n\n{text}\n\nKeywords:"
    result = llm(prompt, max_length=100, do_sample=True, num_return_sequences=1)
    return result[0]["generated_text"].split("\n")[-1]

# 실행 예제
text = "유튜브에서 조회수를 높이는 방법은 최신 트렌드를 분석하고 인기 있는 주제를 찾는 것입니다."
print("Mistral 7B 키워드:", extract_keywords_mistral(text))


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-67cff244-65bb1ac860de04cc002d5632;dd91d5bb-ffb2-4552-8667-02ac5451d1b0)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.